In [26]:
# Import necessary modules
import yaml
from datetime import datetime

In [38]:
import yaml
from datetime import datetime

def generate_ahbl_wrapper(yaml_data):
    ip = yaml_data
    name = ip['info']['name']
    description = ip['info']['description']
    clock = ip['clock']['name']
    reset = ip['reset']['name']

    # Start generating the Verilog code
    verilog_code = f"""\
`default_nettype none

/*
    {name}_AHBL Wrapper
    Description: {description}
    Auto-generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
*/

module {name} (
    // AHB-Lite Interface
    input  wire        HCLK,
    input  wire        HRESETn,
    input  wire [31:0] HADDR,
    input  wire [1:0]  HTRANS,
    input  wire        HWRITE,
    input  wire [2:0]  HSIZE,
    input  wire [31:0] HWDATA,
    input  wire        HSEL,
    input  wire        HREADY,
    output wire [31:0] HRDATA,
    output wire        HREADYOUT,
    output wire        tx
);

    // Register offsets
"""
    # Declare register offsets
    for reg in ip['registers']:
        verilog_code += f"    localparam {reg['name']}_REG_OFF = 32'h{reg['offset']:02X};\n"

    # Address phase signals
    verilog_code += """
    reg [31:0] HADDR_d;
    reg [1:0]  HTRANS_d;
    reg        HWRITE_d, HSEL_d;

    always @(posedge HCLK or negedge HRESETn) begin
        if (~HRESETn) begin
            HADDR_d  <= 32'b0;
            HTRANS_d <= 2'b0;
            HWRITE_d <= 1'b0;
            HSEL_d   <= 1'b0;
        end else if (HREADY) begin
            HADDR_d  <= HADDR;
            HTRANS_d <= HTRANS;
            HWRITE_d <= HWRITE;
            HSEL_d   <= HSEL;
        end
    end

    wire ahbl_we = HTRANS_d[1] & HSEL_d & HWRITE_d;
    wire ahbl_re = HTRANS_d[1] & HSEL_d & ~HWRITE_d;

    // Register Declarations
"""
    # Declare registers
    for reg in ip['registers']:
        verilog_code += f"    reg [{reg['size'] - 1}:0] {reg['name']}_REG;\n"

    # Signal selection logic
    verilog_code += "\n    // Signal Selection Logic\n"
    for reg in ip['registers']:
        verilog_code += f"    wire {reg['name']}_sel = (HADDR_d[23:0] == {reg['name']}_REG_OFF);\n"

    # Register functionality
    verilog_code += "\n    // Register Logic\n"
    for reg in ip['registers']:
        if reg['mode'] in ['rw', 'w']:
            verilog_code += f"""\
    always @(posedge HCLK or negedge HRESETn) begin
        if (~HRESETn)
            {reg['name']}_REG <= {reg.get('init', 0)}; // Default reset value
        else if (ahbl_we & {reg['name']}_sel)
            {reg['name']}_REG <= HWDATA[{reg['size'] - 1}:0];
    end
"""
        elif reg['mode'] == 'r':
            verilog_code += f"    // {reg['name']}_REG is read-only\n"

    # Read data logic
    verilog_code += "\n    // Read Data Logic\n    assign HRDATA =\n"
    for reg in ip['registers']:
        verilog_code += f"        {reg['name']}_sel ? {{32-{reg['size']}'b0, {reg['name']}_REG}} :\n"
    verilog_code += "        32'hBADDBEEF;\n\n"

    # HREADYOUT assignment
    verilog_code += "    assign HREADYOUT = 1'b1;\n\n"

    # Submodule instantiation
    submodule = ip['submodule']
    verilog_code += f"    // Submodule Instance: {submodule['name']}\n"
    verilog_code += f"    {submodule['name']} {submodule['name']}_instance (\n"
    for port in submodule['ports']:
        verilog_code += f"        .{port['name']}({port['connection']}),\n"
    verilog_code = verilog_code.rstrip(",\n") + "\n    );\n"

    # Module end
    verilog_code += "\nendmodule\n"
    return verilog_code

# Load YAML data
yaml_file = "example.yaml"  # Replace with your YAML file path

with open(yaml_file, 'r') as f:
    yaml_data = yaml.safe_load(f)

# Generate Verilog wrapper
verilog_wrapper = generate_ahbl_wrapper(yaml_data)

# Save and display Verilog wrapper
output_file = "ahbl_uart_tx.v"
with open(output_file, 'w') as verilog_file:
    verilog_file.write(verilog_wrapper)

print(verilog_wrapper)


`default_nettype none

/*
    ahbl_uart_tx_AHBL Wrapper
    Description: AHB-Lite Wrapper for UART Transmitter
    Auto-generated on 2024-11-20 04:53:05
*/

module ahbl_uart_tx (
    // AHB-Lite Interface
    input  wire        HCLK,
    input  wire        HRESETn,
    input  wire [31:0] HADDR,
    input  wire [1:0]  HTRANS,
    input  wire        HWRITE,
    input  wire [2:0]  HSIZE,
    input  wire [31:0] HWDATA,
    input  wire        HSEL,
    input  wire        HREADY,
    output wire [31:0] HRDATA,
    output wire        HREADYOUT,
    output wire        tx
);

    // Register offsets
    localparam CTRL_REG_OFF = 32'h00;
    localparam BAUDDIV_REG_OFF = 32'h04;
    localparam STATUS_REG_OFF = 32'h08;
    localparam DATA_REG_OFF = 32'h0C;

    reg [31:0] HADDR_d;
    reg [1:0]  HTRANS_d;
    reg        HWRITE_d, HSEL_d;

    always @(posedge HCLK or negedge HRESETn) begin
        if (~HRESETn) begin
            HADDR_d  <= 32'b0;
            HTRANS_d <= 2'b0;
            HWRITE_d